In [6]:
from Data_interpret.GroundTruth import ReadMat
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import sys

from matplotlib import rcParams
from scipy.interpolate import interp1d
from scipy.integrate import odeint
from information_scores import *
import os
import scipy.io
import pandas as pd
from scipy.io import loadmat
from pathlib import Path
import math
from scipy.misc import derivative
from scipy import signal
from sklearn.linear_model import Lasso

In [7]:
# Loading DATA
train_file = "../Raw_data/8_8_2019 to 8_12_2019/2019-08-08-19-16-02_5sec.mat"
train_file2 = "../Raw_data/8_8_2019 to 8_12_2019/2019-08-09-13-01-42_10sec.mat"
train_data = ReadMat(train_file)
train_data2 = ReadMat(train_file2)
# train_data = ReadMat(train_file2)
test_file = "../Raw_data/8_8_2019 to 8_12_2019/2019-08-09-12-49-56_7sec.mat"
test_data = ReadMat(test_file)

# SAMPLING DATA
sample_size = 100  # sampling of raw data
train_state_traj, train_action_traj = train_data.sampled_mocap_data(sample_size)  # state X time, number of actions X time
train_state_traj2, train_action_traj2 = train_data2.sampled_mocap_data(sample_size)  # state X time, number of actions X time
test_state_traj, test_action_traj = test_data.sampled_mocap_data(sample_size)

In [22]:
# SINDy Function Definitions

def build_library(X,Y):
    
    # obtain size of state space
    l = X.shape[0]
    n = X.shape[1]
    m = Y.shape[1]
    
    # add in a constant by making a column of ones
    Theta = np.ones((l,1))
    function_label = ['1']
    
    # add in first order polynomials of all the states
    for i in range(n):
        state = np.array([X[:,i]]).T
        Theta = np.append(Theta,state,axis=1)
        function_label.append('x'+str(i))
    
    # add in sines of all states 
    for i in range(n):
        sine_state = np.array([np.sin(X[:,i])]).T
        Theta = np.append(Theta,sine_state,axis=1)
        function_label.append('sin x'+str(i))
        
    # add in cosines of all states  
    for i in range(n):
        sine_state = np.array([np.cos(X[:,i])]).T
        Theta = np.append(Theta,sine_state,axis=1)
        function_label.append('cos x'+str(i))
        
    # add in tangents of all states  
    for i in range(n):
        sine_state = np.array([np.tan(X[:,i])]).T
        Theta = np.append(Theta,sine_state,axis=1)
        function_label.append('tan x'+str(i))
        
    
    # add in controls
    for i in range(m):
        control = np.array([Y[:,i]]).T
        Theta = np.append(Theta,control,axis=1) 
        function_label.append('u'+str(i))
 
    return Theta,function_label

def Sparse_Regression(Theta,dXdt,thresh):
    
    lassoreg = Lasso(alpha=thresh,normalize=True, max_iter=1e5) # sparsify!!!!! Hell yeah!!!
    lassoreg.fit(Theta,dXdt)
    Xi = lassoreg.coef_
    
    return Xi.T


In [23]:
dt = 0.01 # sampling time in seconds

X = train_state_traj2.T
Y = train_action_traj2.T
dX = np.zeros((X.shape[0]-4))

for i in range(X.shape[1]):
    Xi = X[:,i]
    dXi = (1/(12*dt)) * (-Xi[4:] + 8*Xi[3:-1] - 8*Xi[1:-3] + Xi[:-4])
    Xi = Xi[2:-2]
    dX = np.vstack([dX,dXi])

Y = Y[2:-2,:]
X = X[2:-2,:]
Xprime = dX[1:,:].T


print(Y.shape,X.shape,Xprime.shape) # print out the shapes of the data matrices as a sanity check

Theta,function_label = build_library(X,Y) # build the lbrary of functions (Theta)
threshold = 0.01 # thresholding paramater for sparsification
Xi = Sparse_Regression(Theta,Xprime,threshold) # obtain Xi ("ksi") that gives us the coefficients
print(Xi.shape)

(1052, 3) (1052, 9) (1052, 9)
(40, 9)


In [25]:
df = pd.DataFrame(Xi,index = function_label)
df.head(40)

,0,1,2,3,4,5,6,7,8
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
x0,-0.000000,-0.000000,-0.000000,0.000000,-0.000000,-0.000000,0.000000,86.794373,-0.000000
x1,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,0.000000
x2,0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000
x3,-0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,27.852783,0.000000
x4,0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
x5,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,9.025253,-0.000000,0.265240
x6,0.000000,0.000000,-0.000000,0.692474,-0.000000,-0.000000,18.810204,1.105953,0.898921
x7,0.000000,0.104809,-0.000000,0.154391,0.667373,0.000000,20.019788,13.840410,0.269278
x8,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,-1.206455,0.000000


In [24]:
function_label

['1',
 'x0',
 'x1',
 'x2',
 'x3',
 'x4',
 'x5',
 'x6',
 'x7',
 'x8',
 'sin x0',
 'sin x1',
 'sin x2',
 'sin x3',
 'sin x4',
 'sin x5',
 'sin x6',
 'sin x7',
 'sin x8',
 'cos x0',
 'cos x1',
 'cos x2',
 'cos x3',
 'cos x4',
 'cos x5',
 'cos x6',
 'cos x7',
 'cos x8',
 'tan x0',
 'tan x1',
 'tan x2',
 'tan x3',
 'tan x4',
 'tan x5',
 'tan x6',
 'tan x7',
 'tan x8',
 'u0',
 'u1',
 'u2']